Data Extraction

In [2]:
import praw
import reddit_keys
from datetime import datetime
import pandas as pd
import numpy as np
import json
import re
import nltk
from nltk.corpus import stopwords

In [3]:
reddit = praw.Reddit(
    client_id="IaZneC0z1RiY95lp_R2H5Q",
    client_secret="VtyTzGdnn97AjC9vDnL3Wb-NvHK_OA",
    user_agent="MDST"
)

In [ ]:
num_posts = 500
posts = []

subreddit_name1 = "politics"
subreddit_name2 = "NeutralPolitics"
subreddit_name3 = "Republican"
subreddit_name4 = "democrats"
subreddit_name5 = "PoliticalDiscussion"

time_filter = "month"
    
subreddit1 = reddit.subreddit(subreddit_name1)
subreddit2 = reddit.subreddit(subreddit_name2)
subreddit3 = reddit.subreddit(subreddit_name3)
subreddit4 = reddit.subreddit(subreddit_name4)
subreddit5 = reddit.subreddit(subreddit_name5) 

subreddits = [subreddit1, subreddit2, subreddit3, subreddit4, subreddit5]

for sr in subreddits:
    for post in sr.top(time_filter=time_filter, limit=num_posts):
        if post.is_self:
            post.comments.replace_more(limit=0)

            comments = post.comments.list()[:50]

            comment_bodies = [comment.body for comment in comments if isinstance(comment, praw.models.Comment)]

            posts.append({
                'title': post.title,
                'content': post.selftext,
                'upvotes': post.score,
                'upvote_ratio': post.upvote_ratio,
                'time_created': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'comments': comment_bodies
            })


/var/folders/8p/n7h1f0bs5p76sqs_h3lhg9ph0000gn/T/ipykernel_15578/2904648029.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'time_created': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


In [4]:
json_filename = 'reddit_politics_posts.json'

In [ ]:
with open(json_filename, 'w') as f:
    json.dump(posts, f, indent=4)

In [ ]:
print(f'Database created and saved to {json_filename}')

In [ ]:
pd.read_json(json_filename)

In [ ]:
# Load the JSON file into a pandas DataFrame
json_filename = 'reddit_politics_posts.json'
df = pd.read_json(json_filename)

# Display the first few rows of the DataFrame
print(df.head())

# Download NLTK stopwords if you haven't already
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Cleaning function from Tutorial.ipynb (adjusted for this context)
def clean_post(post):
    # Convert to lowercase
    post = post.lower()

    # Remove URLs
    post = re.sub(r'http\S+', '', post)

    # Remove special characters, numbers, and punctuations
    post = re.sub(r'[^a-z\s]', '', post)

    # Remove stopwords
    post = ' '.join([word for word in post.split() if word not in stop_words])

    return post

# Cleaning function for lists of comments
def clean_comments(comments):
    # Apply the clean_post function to each comment in the list
    return [clean_post(comment) for comment in comments]

# Apply the cleaning function to both 'content' and 'title' columns
df['cleaned_content'] = df['content'].apply(lambda x: clean_post(str(x)))
df['cleaned_title'] = df['title'].apply(lambda x: clean_post(str(x)))

# Apply the modified cleaning function to the comments
df['cleaned_comments'] = df['comments'].apply(lambda comments: clean_comments(comments))

# Display cleaned data
print(df[['title', 'cleaned_title', 'content', 'cleaned_content']].head())


In [ ]:
df.head()

In [ ]:
df.shape